In [11]:
import uproot3 as uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import os,sys
import math
from sklearn.cluster import Birch#clustering
from scipy import spatial
import scipy.stats as sp

For analysis at h4, we need to
1. Build a sorter, find out index of event with detID between 41-46
2. get their position
3. get info from st23, do linear extrapolation to h4, muon approximately match, pion scatter. <br>
st23tracklet_py_st3<br>
st23tracklet_pz_st3<br>
st23tracklet_px_st3<br>

Double_t track_x_CAL = track_x_st3 + (track_px_st3 / track_pz_st3) * (1930. - track_z_st3);<br>
Double_t track_y_CAL = track_y_st3 + (track_py_st3 / track_pz_st3) * (1930. - track_z_st3);<br>
st23tracklet_x_st3<br>
st23tracklet_y_st3<br>
st23tracklet_z_st3<br>

For most particles, their hits ID between 41, 46 record are more tham recorded hits on h4. Since hit_pos only provided with detID, i will use detID method to find out evt that has hits on h4

In [44]:
#--------------------------------------------------------------------------------------------------------------------------------------------
def getData(fname="", procName="Events"):
    kn_dict = uproot.open(fname)["Events"].arrays()
    kn_dict_ak1 = {name.decode(): ak.from_awkward0(array) for name, array in kn_dict.items()}
    kn_events = ak.zip({"Electrons":ak.zip({
                                            "ge":      kn_dict_ak1["ge"],
                                            "gvx":      kn_dict_ak1["gvx"],
                                            "gvy":      kn_dict_ak1["gvy"],
                                            "gvz":      kn_dict_ak1["gvz"],
                                            "gpx":      kn_dict_ak1["gpx"],
                                            "gpy":      kn_dict_ak1["gpy"],
                                            "gpz":      kn_dict_ak1["gpz"],
                                        }),
                        "Hits":ak.zip({
                                      "detID":   kn_dict_ak1["hit_detID"],#detector ID for each hit, look for 41-46
                                      "edep":    kn_dict_ak1["hit_edep"],
                                      "elmID":   kn_dict_ak1["hit_elmID"],#EMCal cell ID
                                      "hit_pos":  kn_dict_ak1["hit_pos"],#then find the position of hits(use index) with ID bet41-46
                                        }),
                        "track":ak.zip({
                                      "x":   kn_dict_ak1["track_x_CAL"],
                                      "y":    kn_dict_ak1["track_y_CAL"],
                                      "ID":    kn_dict_ak1["eventID"],
                                      "pz":    kn_dict_ak1["track_pz_st1"],
                                        }),
                        "st23":ak.zip({
                                      "ntrack23":   kn_dict_ak1["n_st23tracklets"],
                                      "px":   kn_dict_ak1["st23tracklet_px_st3"],
                                      "py":   kn_dict_ak1["st23tracklet_py_st3"],
                                      "pz":   kn_dict_ak1["st23tracklet_pz_st3"],
                                      "x":   kn_dict_ak1["st23tracklet_x_st3"],
                                      "y":   kn_dict_ak1["st23tracklet_y_st3"],
                                      "z":   kn_dict_ak1["st23tracklet_z_st3"],
                                        }),
                       }, depth_limit=1)
    return kn_events

In [264]:
def h4_sort(raw_ID, raw_pos):#hard coded for detID 41-46
    indices = []
    values = []
    h4_pos = []
    for index, value in enumerate(raw_ID):
        if 41 <= value <= 46:
            indices.append(index)
            values.append(value)
            h4_pos.append(raw_pos[index])
    return indices, values, h4_pos

In [265]:
def extrpl_h4(x,y,z,px,py,pz,ID):#41-44 are x. 45, 46 are y.
    IDpos={41: 2130.27, 42: 2146.45, 43:2200.44 , 44:2216.62 , 45:2251.71 , 46: 2234.29}
    h4_x = x+(px/pz)*(IDpos[ID]-z)
    h4_y = y+(py/pz)*(IDpos[ID]-z)
    return h4_x, h4_y

In [278]:
os.chdir("/Users/wongdowling/Desktop/DQ_Dowling/ntuple_1000")
file="electron_520_1000eve.root"
dq_events = getData(file)
ID = dq_events["Hits"].detID.tolist()
raw_pos = dq_events["Hits"].hit_pos.tolist()
st3_x= dq_events["st23"].x.tolist()#st3 may catch multiple track or no track
st3_y= dq_events["st23"].y.tolist()
st3_z= dq_events["st23"].z.tolist()
st3_px= dq_events["st23"].px.tolist()
st3_py= dq_events["st23"].py.tolist()
st3_pz= dq_events["st23"].pz.tolist()

pos_3D=[]#event ->h4 hit ->each track
hit_pos=[]

for i in range(len(ID)):#in level of each event
    indices, values, pos_1D=h4_sort(ID[i], raw_pos[i])
    hit_pos.append(pos_1D)
    x=st3_x[i]
    y=st3_y[i]
    z=st3_z[i]
    px=st3_px[i]
    py=st3_py[i]
    pz=st3_pz[i]
    pos_2D=[]
    for item in values:
        pos=[]
        for j in range(len(x)):
            pos.append(extrpl_h4(x[j], y[j], z[j], px[j], py[j], pz[j], item))
        pos_2D.append(pos)
    pos_3D.append(pos_2D)

    

In [279]:
len(hit_pos[12])

0

In [281]:
N=0
for item in st3_x:
    if len(item)!=1:
        N+=1
        print(item)
        

[3.9360644817352295, 8.181635856628418]
[3.071784496307373, 9.688541412353516]
[-3.7286477088928223, -0.3143143653869629]
[8.11592960357666, 20.32457733154297, 6.556115627288818]
[]
[]
[0.5874420404434204, -22.52470588684082]
[11.169495582580566, 3.70550799369812, 51.16697692871094]
[]
[]
[12.791733741760254, 20.955698013305664]
[-34.24217987060547, -25.557456970214844]
[]
[-1.9390957355499268, 33.11975860595703]
[]
[-1.8209927082061768, -20.765073776245117]
[3.9710750579833984, 9.896692276000977, 16.449745178222656]
[-10.129376411437988, -15.220988273620605, 6.1236114501953125]
[9.551346778869629, -6.060884475708008]
[-0.7344173192977905, -9.765981674194336]
[3.264235496520996, -51.20107650756836]
[6.811129570007324, 47.57767105102539, 6.520510196685791, 47.82097244262695]
[]
[11.260603904724121, 12.357321739196777]
[6.376445770263672, -12.732553482055664, 6.328092098236084]
[]
[9.897176742553711, 39.052249908447266]
[-4.018930912017822, -60.83573532104492, -3.5057177543640137]
[-26.6

In [282]:
N

66

In [234]:
ex_pos_3D=[]
for item in result:
    pos_2D=[]
    for element in enumerate(item):
        x=st3_x[element[0]]
        y=st3_y[element[0]]
        z=st3_z[element[0]]
        px=st3_px[element[0]]
        py=st3_py[element[0]]
        pz=st3_pz[element[0]]
        pos=[]
        for i in range(len(x)):
            pos.append(extrpl_h4(x[i], y[i], z[i], px[i], py[i], pz[i], element[1]))
        pos_2D.append(pos)
    ex_pos_3D.append(pos_2D)
        
        
        

In [241]:
len(ID)

1000